<a href="https://colab.research.google.com/github/JanEggers-hr/chatgpt-playground/blob/main/konvertiere_bilder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bilder beschreiben

Ein Skript, das aus Bildern zu den DMT Beschreibungen extrahiert: Die Bilder werden an GPT-4V übergeben, mit der Anweisung, sie genau zu beschreiben.

Die Bilder werden zunächst im **Google-Drive** des Anwenders in ein Verzeichnis ```MyDrive/bilder-konvertieren``` geladen; wenn es genug sind, einfach den "Konvertieren..."-Knopf klicken.

Über die API wird dann bei GPT-4 eine Bildbeschreibung angefordert - das Prompt, mit dem das passiert, kann man vor Beginn der Konvertierung anpassen. Die Ergebnisse landen in einer Excel-Tabelle und werden dann am Ende als Datei ```bildbeschreibungen.xlsx``` heruntergeladen.

Bilder, die erfolgreich konvertiert sind, am besten **von Hand wieder aus dem Google-Drive-Verzeichnis löschen.**

## **Was man braucht:**

- ein Google-Konto, um das Colab auszuführen;
- ein OpenAI-API-Token mit Zugangsberechtigung zu GPT-4.

Einfach unten nacheinander auf die "Play"-Schaltflächen bei Block 1 und Block 2 klicken, um die Ausführung zu starten.

jan.eggers at hr.de

In [ ]:
#@title Block 1: Vorbereitung
import requests
import json
import math
import markdown

# ipywidgets ist schon installiert
import ipywidgets as widgets
from IPython.display import display

# Modelle und Kosten definieren
# Kosten in US-Dollar je 1000 Tokens
# Könnte die Modelle auch über die API holen, aber so kann ich die Kosten
# mitgeben. Entsprechend OpenAI-Preisliste Dezember 2023. https://openai.com/pricing
# Output ist inzwischen doppelt so teuer wie Input. Hier werden die Output-
# Preise zur Berechnung genutzt.
models_token_info = {
          'gpt-4-1106-preview': {
                                        'output_price': 0.03,
                                        'input_price': 0.01,
                                        'max_tokens': 128000
                                      },
          'gpt-3.5-turbo-1106': {
                                        'output_price': 0.002,
                                        'input_price': 0.001,
                                        'max_tokens': 16385
                                      },
          'gpt-4-vision-preview': {
                                        'output_price': 0.03,
                                        'input_price': 0.01,
                                        'max_tokens': 128000
                                      },
          }



# Tokenizer Tiktoken einbinden
!pip install -q tiktoken
import tiktoken
print("Tokenizer tiktoken geladen.")

# OpenAI-API-Library einbinden
!pip install -q openai
from openai import OpenAI
print("OpenAI-API-Library geladen.")

##### Der eigentliche Code! #####

from getpass import getpass
from google.colab import userdata

key_needed = True
try:
  ai_key_name = userdata.get('openai')
  ai_client = OpenAI(api_key = userdata.get(ai_key_name))
except:
  print("OpenAI-Key benötigt")
else:
  print("*** API-Key gültig, KI einsatzbereit! ***")
  key_needed = False

while key_needed:
    try:
        # Testweise Modelle abfragen
        ai_client = OpenAI(api_key = getpass("OpenAI-API-Key eingeben: "))
        models = ai_client.models.list()
        # Returns a list of model objects
        # Erfolg?
        print("API-Key gültig!")
        key_needed = False
    except Exception as e:
        print("Fehler bei Abfrage; ist der API-Key möglicherweise ungültig?", e)

previous_messages = []
spent_tokens = 0        # Wie viele Tokens wurden bisher über die API abgefragt?
spent_dollars = 0.00    # Zu welchem Preis?
codeblock = False       # Hat Ausgabe eines Codeblocks begonnen?

# Define the HTML and JavaScript code for the spinner animation
spinner_html = """
<div class="loader"></div>
<style>
.loader {
  border: 8px solid #f3f3f3;
  border-top: 8px solid #3498db;
  border-radius: 50%;
  width: 25px;
  height: 25px;
  animation: spin 2s linear infinite;
  margin: 20px auto;
}

@keyframes spin {
  0% { transform: rotate(0deg); }
  100% { transform: rotate(360deg); }
}
</style>
"""


# Funktion, die das Sprachmodell direkt anzapft (für Preprocessing und Kategorienbildung)
# Braucht: Systemprompt, Beispiele (als User-Assistant-Dialog), Modell
# Gibt zurück: einen String
# Updated die verbrauchten Tokens.

# Hilfsfunktion: Token berechnen
def calculate_tokens(string: str) -> int:
    """Returns the number of tokens in a text string."""
    # cl100k_base ist der Tokenizer für Davinci, GPT-3 und GPT-4
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

def output_pricing(tokens,model):
    price = models_token_info.get(model)['output_price']
    # Kosten in Dollar zurückgeben
    return(tokens * price / 1000)

def input_pricing(tokens,model):
    price = models_token_info.get(model)['input_price']
    # Kosten in Dollar zurückgeben
    return(tokens * price / 1000)

def gpt(g_system,g_previous_messages,g_prompt,g_model,temperature = 0):
    # Systemprompt und few-shots zusammenbinden
    #
    model = g_model
    global max_tokens
    global stoptokens
    global spent_tokens
    global spent_dollars
    if json==True:
        m_type = {"type": "json_object"}
    else:
        m_type = {"type": "text"}

    prompts = [
            {"role": "system", "content": g_system},
            *g_previous_messages,
            {"role": "user", "content": g_prompt},
        ]
    response = ai_client.chat.completions.create(
        messages=prompts,
        model=g_model,
        max_tokens=max_tokens,
        temperature=temperature,
    )
    # Anzahl verbrauchtert Tokens anpassen
    spent_tokens += response.usage.total_tokens
    spent_dollars += output_pricing(response.usage.completion_tokens,g_model) + input_pricing(response.usage.prompt_tokens,g_model)
    token_usage_text = f'<b>Verbrauchte Token:</b> {spent_tokens} ($ {spent_dollars:.3f}) '
    return(response.choices[0].message.content)





In [ ]:
#@title Block 2: Daten laden

from google.colab import files, drive
# Connect to Google Drive to export data
import os
import io
import pandas as pd
import glob
import base64
import requests
import openpyxl
from io import BytesIO
from PIL import Image as PILImage
from openpyxl.drawing.image import Image
from openpyxl.styles import Alignment


drive.mount('/content/gdrive')

# Create output directory
output_dir = "/content/gdrive/MyDrive/bilder-konvertieren"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

def list_files():
    file_types = ['*.jpg', '*.png', '*.webp', '*.gif']

    # Initialize count
    dateien_liste = []

    # Iterate over each file type and count files
    for file_type in file_types:
        pic_files = glob.glob(os.path.join("./", file_type))
        dateien_liste.extend(pic_files)
    return dateien_liste

def count_files():
    dateien_liste = list_files()
    count = len(dateien_liste)
    return count

os.chdir(output_dir)
number_files = count_files()

html_dateien = widgets.HTML (
    value = f"<p>Anzahl geladener Bilddateien: <strong>{number_files}</strong></p>"
)

button_hochladen = widgets.Button (
    description='Hochladen',
    layout=widgets.Layout(width='15%'),
)

button_konvertieren = widgets.Button (
    description='Konvertieren',
    layout=widgets.Layout(width='15%'),
)

html_status = widgets.HTML(
    value='''<table><tr>
      <td>pic</td>
      </tr><tr>
        <td>description</td>
      </tr></table>
    ''',
    layout=widgets.Layout(width='100%')
)

# Dateien dazu laden
def upload_pics(value):
    uploaded = files.upload()
    number_files = count_files()
    html_dateien.value = f"<p>Anzahl geladener Bilddateien: <strong>{number_files}</strong></p>"


# Dateien in Base64 kodieren:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

def generate_table_row(base64_image, text):
  s = f'''<tr>
      <td><img src="data:image/jpeg;base64,{base64_image}" width=200></td>
      <td>{text}</td>
      </tr>'''
  return s

def set_status(base64_image,text,previous_rows):
  new_row = generate_table_row(base64_image, text)
  html_status.value = f'''<p>Verbrannte Tokens: <strong>{spent_tokens} (Dollar: $ {spent_dollars})</strong></p>
  <table border ="1"><tr>
      <th>Bild</th><th>Beschreibung</th></tr>
      {new_row}{previous_rows}</table>
    '''
  return new_row

# Bilder in das Excel laden
def pictify(f):
  wb = openpyxl.load_workbook(f)
  ws = wb.active
  # Spalte B enthält die Dateipfade, C die Bilder, D den Alt-Text
  filename_column = 2
  text_column = 3
  wrap_text = Alignment(wrap_text=True)
  # Spaltenbreite setzen
  ws.column_dimensions['C'].width = 15
  ws.column_dimensions['D'].width = 15
  for row in range(2, ws.max_row + 1):
    cell = ws.cell(row=row, column=text_column)
    cell.alignment = wrap_text
    # Read the filename
    filename = ws.cell(row=row, column=filename_column).value
    if os.path.exists(filename):
        # Bild in Thumbnail
        pil_image = PILImage.open(filename)
        pil_image.thumbnail((512, 384), PILImage.ANTIALIAS)

        # Save the resized image to a BytesIO object
        image_stream = BytesIO()
        pil_image.save(image_stream, format='png')
        image_stream.seek(0)
        img = Image(image_stream)
        ws.add_image(img, f'D{row}')  # Adjust cell as needed
        # Zellengröße anpassen
        ws.row_dimensions[row].height = 512  # height in points
  wb.save(f)



# Dateien aus dem Verzeichnis umwandeln:
def convert_pics(value):
    # Status-HTML anzeigen
    display(html_status)
    dateien_liste = list_files()
    images_liste = []
    results_liste = []
    old_rows = ""
    for f in dateien_liste:
        base64_image = encode_image(f)
        set_status(base64_image,spinner_html,old_rows)
        system_prompt = area_system.value
        picture_prompt = [{
      "role": "user",
      "content": [
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}",
            "detail": "high",
          }
        }
      ]
    }]
        result = gpt(system_prompt,
            picture_prompt,
                     "Beschreibe dieses Bild präzise.",
            g_model = "gpt-4-vision-preview",
            temperature = 0.0)
        old_rows = old_rows + set_status(base64_image,result,old_rows)
        images_liste.append(f"data:image/jpeg;base64,{base64_image}")
        results_liste.append(result)
    df = pd.DataFrame({
    'Dateien': dateien_liste,
    'Alt-Text': results_liste,
    'Bilder': images_liste
     })
    df.to_excel("bildbeschreibungen.xlsx")
    pictify("bildbeschreibungen.xlsx")
    files.download("bildbeschreibungen.xlsx")


button_hochladen.on_click(upload_pics)
button_konvertieren.on_click(convert_pics)


bild_prompt = '''Du bist Barrierefreiheits-Redakteur. Deine Aufgabe ist, \
Bilder so zu beschreiben, dass blinde Menschen eine vollständige Inhaltsangabe \
über das Alt-Tag bekommen können.
Die Bilder enthalten Informationen zu einer der folgenden Nutzergruppen jeweils \
im Vergleich zu den soziodemographischen Daten der Grundgesamtheit:
- Vereinfachende
- Ablenkungssuchende
- Ambitionierte
- Neokulturelle
- Selbstgenügsame
- Sicherheitsorientierte
- Traditionsbewusste
- Verantwortungsvolle
- Gleichmütige
Beschreibe zunächst, um welchen dieser neun Nutzertypen es sich handelt. \
Beschreibe dann detailliert alle Grafiken. Zitiere alle Texte und alle Zahlen.
Nutze Markdown, Zwischenüberschriften und Zeilenumbrüche, um den Text zu gliedern.
'''

area_system = widgets.Textarea(
    value = bild_prompt,
    rows=10,
    description = 'Prompt:',
    layout=widgets.Layout(width='80%')
)

# Globale Werte:
max_tokens = 2048

# Jetzt Display
display(button_hochladen)
display(html_dateien)
display(area_system)
display(button_konvertieren)


